In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats  # Required for T-tests, Chi-Squared, ANOVA

# Load your cleaned data (ensure path is correct relative to the notebooks folder)
try:
    df = pd.read_csv('../data/MachineLearningRating_v3.txt', sep='|') # or the .csv if you saved a clean version
    
    # Quick fix for column names just in case
    df.columns = df.columns.str.strip().str.replace('[^A-Za-z0-9_]+', '_', regex=True).str.lower()
    
    print("Data loaded and columns cleaned.")
except FileNotFoundError:
    print("Error: File not found. Check your path.")

C:\Users\Her\AppData\Local\Temp\ipykernel_35860\124543369.py:9: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/MachineLearningRating_v3.txt', sep='|') # or the .csv if you saved a clean version


Data loaded and columns cleaned.


In [2]:
# --- 1. Address DtypeWarning and Final Cleaning ---

# Re-load the data and explicitly set low_memory=False to let pandas determine types better
# We keep the column cleanup just in case.
file_path = '../data/MachineLearningRating_v3.txt'
df = pd.read_csv(file_path, sep='|', low_memory=False)

# Re-apply the column cleanup
df.columns = df.columns.str.strip().str.replace('[^A-Za-z0-9_]+', '_', regex=True).str.lower()

# Columns 32 and 37 are likely 'calculatedpremiumperterm' and 'suminsured'
# We will coerce all critical financial columns to numeric, converting errors to NaN
financial_cols = ['totalpremium', 'totalclaims', 'customvalueestimate', 'calculatedpremiumperterm', 'suminsured']

for col in financial_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop any rows that became entirely invalid (optional, but good practice)
df.dropna(subset=financial_cols, inplace=True)

print("Data cleaning complete. Financial columns converted to numeric.")

Data cleaning complete. Financial columns converted to numeric.


In [3]:
# --- 2. Create the Required KPIs ---

# 1. Claim Frequency (The 'claimed' binary column)
# If totalclaims > 0, set to 1. Otherwise, set to 0.
df['claimed'] = np.where(df['totalclaims'] > 0, 1, 0)

# 2. Margin (Profit Calculation)
# Margin = TotalPremium - TotalClaims
df['margin'] = df['totalpremium'] - df['totalclaims']

# 3. Prepare Data for Claim Severity
# Create the subset of data for testing severity hypotheses.
df_severity = df[df['claimed'] == 1].copy()

# --- Verification ---
print("\nKPIs Created:")
print(df[['totalpremium', 'totalclaims', 'claimed', 'margin']].describe().T[['count', 'mean', 'min', 'max']])
print(f"\nTotal Policies (df): {len(df)}")
print(f"Policies with Claims (df_severity): {len(df_severity)}")


KPIs Created:
                 count       mean            min            max
totalpremium  220456.0  74.813039     -80.409357    1538.957719
totalclaims   220456.0  74.989311       0.000000  363343.421053
claimed       220456.0   0.002989       0.000000       1.000000
margin        220456.0  -0.176272 -362278.393070    1538.957719

Total Policies (df): 220456
Policies with Claims (df_severity): 659


In [4]:
# Create the contingency table: Gender vs. Claimed (0 or 1)
contingency_table = df.groupby(['gender', 'claimed']).size().unstack(fill_value=0)

# Ensure both 0 (Not Claimed) and 1 (Claimed) columns exist, though they usually will
if 1 not in contingency_table.columns:
    contingency_table[1] = 0
if 0 not in contingency_table.columns:
    contingency_table[0] = 0

# Select only the Claimed column for the test (Optional: Can use the whole table for chi2)
# We use the full table for the standard Chi-Squared test
chi2, p_value_freq, dof, expected = stats.chi2_contingency(contingency_table)

print("--- Claim Frequency (Gender) ---")
print(f"Observed Frequencies:\n{contingency_table}")
print(f"Chi-Squared Statistic: {chi2:.4f}")
print(f"P-Value: {p_value_freq:.4f}")

--- Claim Frequency (Gender) ---
Observed Frequencies:
claimed             0    1
gender                    
Female            115    1
Male             3997   12
Not specified  213651  642
Chi-Squared Statistic: 1.2269
P-Value: 0.5415


In [5]:
# Filter severity data into two groups: Women and Men
men_claims = df_severity[df_severity['gender'] == 'men']['totalclaims']
women_claims = df_severity[df_severity['gender'] == 'women']['totalclaims']

# Perform the independent two-sample t-test (assuming unequal variance, which is safe)
t_stat_severity, p_value_severity = stats.ttest_ind(
    men_claims, 
    women_claims, 
    equal_var=False # Assumes variances are not equal
)

print("\n--- Claim Severity (Gender) ---")
print(f"Men Average Claim: {men_claims.mean():.2f}")
print(f"Women Average Claim: {women_claims.mean():.2f}")
print(f"T-Statistic: {t_stat_severity:.4f}")
print(f"P-Value: {p_value_severity:.4f}")


--- Claim Severity (Gender) ---
Men Average Claim: nan
Women Average Claim: nan
T-Statistic: nan
P-Value: nan


c:\Users\Her\Desktop\Week-3\.venv\Lib\site-packages\scipy\_lib\deprecation.py:234: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  return f(*args, **kwargs)


In [6]:
# --- 1. Claim Frequency across all Provinces (Chi-Squared) ---
# Assuming your province column is named 'province'

# Create contingency table: Province vs. Claimed (0 or 1)
province_freq_table = df.groupby(['province', 'claimed']).size().unstack(fill_value=0)

chi2_province_freq, p_value_province_freq, dof_province_freq, expected_province_freq = stats.chi2_contingency(province_freq_table)

print("\n--- Claim Frequency (Province) ---")
print(f"Chi-Squared Statistic: {chi2_province_freq:.4f}")
print(f"P-Value: {p_value_province_freq:.4f}")


--- Claim Frequency (Province) ---
Chi-Squared Statistic: 26.5474
P-Value: 0.0008


In [7]:
# --- 2. Claim Severity across all Provinces (ANOVA) ---
# Requires filtering the severity data by province

# Identify all provinces in the severity subset
province_groups = [
    df_severity[df_severity['province'] == prov]['totalclaims']
    for prov in df_severity['province'].unique()
]

# Run ANOVA test to compare the means of totalclaims across all provinces
f_stat_province_severity, p_value_province_severity = stats.f_oneway(*province_groups)

print("\n--- Claim Severity (Province) ---")
print(f"F-Statistic: {f_stat_province_severity:.4f}")
print(f"P-Value: {p_value_province_severity:.4f}")


--- Claim Severity (Province) ---
F-Statistic: 1.4913
P-Value: 0.1568


In [8]:
# --- ZIP CODE HYPOTHESIS TEST PREP ---

# 1. Filter to high volume zip codes (Adjust threshold as needed, e.g., 50)
ZIP_THRESHOLD = 50
zip_counts = df['postalcode'].value_counts()
high_volume_zips = zip_counts[zip_counts >= ZIP_THRESHOLD].index

df_filtered = df[df['postalcode'].isin(high_volume_zips)]

In [9]:
# 2. Identify High Risk and Low Risk Groups based on your EDA (use mean Claimed or mean Margin)
zip_summary = df_filtered.groupby('postalcode').agg(
    ClaimRate=('claimed', 'mean'),
    AvgMargin=('margin', 'mean')
).sort_values(by='ClaimRate', ascending=False)

# Select the top and bottom 10 for comparison
top_risky_zips = zip_summary.head(10).index
low_risky_zips = zip_summary.tail(10).index

# Create test groups
high_risk_group = df_filtered[df_filtered['postalcode'].isin(top_risky_zips)]
low_risk_group = df_filtered[df_filtered['postalcode'].isin(low_risky_zips)]

In [10]:
# --- 3. Hypothesis 2: Claim Severity (Risk Difference) ---
# Filter to policies with claims in the high/low risk groups for severity testing
high_risk_severity = high_risk_group[high_risk_group['claimed'] == 1]['totalclaims']
low_risk_severity = low_risk_group[low_risk_group['claimed'] == 1]['totalclaims']

t_stat_zip_severity, p_value_zip_severity = stats.ttest_ind(
    high_risk_severity, 
    low_risk_severity, 
    equal_var=False
)

print("\n--- Claim Severity (Zip Code - T-Test High vs. Low) ---")
print(f"P-Value: {p_value_zip_severity:.4f}")


--- Claim Severity (Zip Code - T-Test High vs. Low) ---
P-Value: nan


c:\Users\Her\Desktop\Week-3\.venv\Lib\site-packages\scipy\_lib\deprecation.py:234: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  return f(*args, **kwargs)


In [11]:
# --- 4. Hypothesis 3: Margin (Profit Difference) ---

# Compare the average margin between the two groups
t_stat_zip_margin, p_value_zip_margin = stats.ttest_ind(
    high_risk_group['margin'], 
    low_risk_group['margin'], 
    equal_var=False
)

print("\n--- Margin Difference (Zip Code - T-Test High vs. Low) ---")
print(f"P-Value: {p_value_zip_margin:.4f}")


--- Margin Difference (Zip Code - T-Test High vs. Low) ---
P-Value: 0.0034


In [12]:
# --- ZIP CODE HYPOTHESIS 2 (Severity) - REVISED T-TEST ---

# Group 1: High Risk Zip Codes (already defined)
high_risk_severity = high_risk_group[high_risk_group['claimed'] == 1]['totalclaims']

# Group 2: The rest of the policies with claims (Control Group)
# Filter main DF to claims, then exclude the high_risk_zips
df_severity_control = df_severity[~df_severity['postalcode'].isin(top_risky_zips)]
control_severity = df_severity_control['totalclaims']

# Run the T-test again
t_stat_zip_severity_revised, p_value_zip_severity_revised = stats.ttest_ind(
    high_risk_severity, 
    control_severity, 
    equal_var=False 
)

print("\n--- Claim Severity (Zip Code - Revised T-Test High vs. All Others) ---")
print(f"P-Value: {p_value_zip_severity_revised:.4f}")


--- Claim Severity (Zip Code - Revised T-Test High vs. All Others) ---
P-Value: 0.8054


In [13]:
# --- GENDER HYPOTHESIS 4 (Severity) - REVISED T-TEST ---

# Check which group is larger (likely Men) and compare it against the rest.
# Let's compare Men vs. Women + Not Specified claims.

men_claims = df_severity[df_severity['gender'] == 'men']['totalclaims']
non_men_claims = df_severity[df_severity['gender'] != 'men']['totalclaims']

# Run the independent two-sample t-test 
t_stat_gender_severity_revised, p_value_gender_severity_revised = stats.ttest_ind(
    men_claims, 
    non_men_claims, 
    equal_var=False
)

print("\n--- Claim Severity (Gender - Revised T-Test Men vs. Others) ---")
print(f"P-Value: {p_value_gender_severity_revised:.4f}")


--- Claim Severity (Gender - Revised T-Test Men vs. Others) ---
P-Value: nan
